In [1]:
from dask.distributed import Client

client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 72,Total memory: 125.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38033,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 72
Started: Just now,Total memory: 125.47 GiB
Comm: tcp://127.0.0.1:35557,Total threads: 18
Dashboard: /proxy/38129/status,Memory: 31.37 GiB
Nanny: tcp://127.0.0.1:38709,


In [6]:
import xarray as xr
import os
import numpy as np

In [12]:
def load_data():
    """
    Load era5 2-meter daily average temperature data over Antarctica
    
    # era5dir = "/glade/work/zespinosa/data/era5/daily/2m-T/nh_60N-90N_195901-202212/anoms"
    # dfiles = [os.path.join(era5dir, f"anoms-2m-T_{dec}_60N-90N.nc") for dec in decades]
    """
    era5dir = "/glade/work/zespinosa/data/era5/daily/2m-T/sh_60S-90S_195901-202212/"

    era5_2mT = xr.open_dataset(os.path.join(era5dir, "2m-T_195901-202212_50S-90S.nc"), chunks={"lat": 50, "lon": 50, "time": -1})
    era5_2mT = era5_2mT.sel(time=np.arange("1980-01", "2023-01", dtype="datetime64[D]"))
    return era5_2mT.t2m

# Load era5 2m-T data
era5_2mT = load_data()

# Load land-sea-mask
landmask = xr.open_dataset("/glade/work/zespinosa/GRIDS/era5_land_sea_mask_25x25.nc")
landmask = landmask.sel(lat=era5_2mT.lat, lon=era5_2mT.lon)

# Mask out ocean in era5
era5_2mT = era5_2mT.where(landmask.lsm, np.nan)
era5_2mT

<xarray.DataArray 't2m' (time: 15706, lat: 161, lon: 1440)>
dask.array<where, shape=(15706, 161, 1440), dtype=float32, chunksize=(15706, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) datetime64[ns] 1980-01-01 ... 2022-12-31
  * lon                      (lon) float64 -180.0 -179.8 -179.5 ... 179.5 179.8
  * lat                      (lat) float64 -90.0 -89.75 -89.5 ... -50.25 -50.0
    realization              int64 ...
    experimentVersionNumber  object ...
Attributes:
    standard_name:          air_temperature
    long_name:              2 metre temperature
    units:                  K
    comment:                near-surface (usually, 2 meter) air temperature
    cds_magics_style_name:  near-surface-air-temperature
    type:                   real

In [13]:
era5_2mT.load()

<xarray.DataArray 't2m' (time: 15706, lat: 161, lon: 1440)>
array([[[244.4138 , 244.4138 , 244.4138 , ..., 244.4138 , 244.4138 ,
         244.4138 ],
        [245.51056, 245.51366, 245.51617, ..., 245.50243, 245.50487,
         245.50795],
        [246.47289, 246.47816, 246.4837 , ..., 246.45595, 246.46141,
         246.4671 ],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[243.23413, 243.23413, 243.23413, ..., 243.23413, 243.23413,
         243.23413],
        [244.30371, 244.30713, 244.31038, ..., 244.29314, 244.29663,
         244.30022],
        [245.47185, 245.47876, 245.4856 , ..., 245.45134, 245.45825,
         245.46501],
...
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[246.08167, 246.08167, 246.08167, ..., 246.08167, 246.08167,
         246.08167],
        [245.83679, 245.83899, 245.84135, ..., 245.83044, 245.83257,
         245.83452],
        [245.00972, 245.01411, 245.0186 , ..., 244.99646, 245.00093,
         245.00525],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]]], dtype=float32)
Coordinates:
  * time                     (time) datetime64[ns] 1980-01-01 ... 2022-12-31
  * lon                      (lon) float64 -180.0 -179.8 -179.5 ... 179.5 179.8
  * lat                      (lat) float64 -90.0 -89.75 -89.5 ... -50.25 -50.0
    realization              int64 0
    experimentVersionNumber  <U4 '0001'
Attributes:
    standard_name:          air_temperature
    long_name:              2 metre temperature
    units:                  K
    comment:                near-surface (usually, 2 meter) air temperature
    cds_magics_style_name:  near-surface-air-temperature
    type:                   real

In [14]:
era5_2mT.std(["time"])

/glade/work/zespinosa/conda-envs/cenv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


<xarray.DataArray 't2m' (lat: 161, lon: 1440)>
array([[11.121735 , 11.121735 , 11.121735 , ..., 11.121735 , 11.121735 ,
        11.121735 ],
       [11.1278515, 11.12783  , 11.127796 , ..., 11.128001 , 11.127969 ,
        11.127904 ],
       [10.937031 , 10.936964 , 10.936878 , ..., 10.937337 , 10.937227 ,
        10.937136 ],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * lon                      (lon) float64 -180.0 -179.8 -179.5 ... 179.5 179.8
  * lat                      (lat) float64 -90.0 -89.75 -89.5 ... -50.25 -50.0
    realization              int64 0
    experimentVersionNumber  <U4 '0001'

In [ ]:
# Good practice to close any Dask cluster
client.shutdown()